<a href="https://colab.research.google.com/github/Tclack88/misc-analysis/blob/main/vietnamese_frequency_sentence_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar"

--2022-01-09 23:08:03--  https://github.com/qhungngo/EVBCorpus/raw/master/EVBCorpus_EVBNews_v2.0.rar
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar [following]
--2022-01-09 23:08:03--  https://raw.githubusercontent.com/qhungngo/EVBCorpus/master/EVBCorpus_EVBNews_v2.0.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7310993 (7.0M) [application/octet-stream]
Saving to: ‘EVBCorpus_EVBNews_v2.0.rar’

EVBCorpus_EVBNews_v 100%[===================>]   6.97M  --.-KB/s    in 0.1s    

2022-01-09 23:08:04 (69.9 MB/s) - ‘EVBCorpus_EVBNews_v2.0.rar’ s

In [2]:
!unrar e "EVBCorpus_EVBNews_v2.0.rar"  &> /dev/null 

In [2]:
!pip install pandas==1.3.0
from pathlib import Path
import pandas as pd
pd.__version__

'1.3.0'

In [ ]:
# !pip install sgmllib3k
# import sgmllib
# class FoundTitle(Exception):
#     pass

# class ExtractTitle(sgmllib.SGMLParser):

#     def __init__(self, verbose=0):
#         sgmllib.SGMLParser.__init__(self, verbose)
#         self.title = self.data = None

#     def handle_data(self, data):
#         if self.data is not None:
#             self.data.append(data)

#     def start_title(self, attrs):
#         self.data = []

#     def end_title(self):
#         self.title = "".join(self.data)
#         raise FoundTitle # abort parsing!

# def extract(file):
#     # extract title from an HTML/SGML stream
#     p = ExtractTitle()
#     try:
#         while 1:
#             # read small chunks
#             s = file.read(512)
#             if not s:
#                 break
#             p.feed(s)
#         p.close()
#     except FoundTitle:
#         return p.title
#     return None
    
# extract(open("N0001.sgml"))

In [55]:
from bs4 import BeautifulSoup
import re

def process(instring): # remove punctuation and make lowercase
  outstring = re.sub(r'[^\w\s]', '', instring)
  return outstring.lower()  

sgml_files = Path('.').glob('*.sgml')
en = []
vn = []
for file in sgml_files:
  with open(file,'r') as f:
    doc = f.read()
    soup = BeautifulSoup(doc, 'html.parser')
    out = soup.find_all('spair')
    for i in range(len(out)):
      e = process(out[i].find('s', {'id':f'en{i+1}'}).getText())
      v = process(out[i].find('s', {'id':f'vn{i+1}'}).getText())
      if len(v.split()) <= 12:
        if v not in vn: # avoid duplicates
          en.append(e)
          vn.append(v)

In [56]:
print(len(vn), len (en))

9077 9077


In [57]:
freq = pd.read_html("https://1000mostcommonwords.com/1000-most-common-vietnamese-words/", header=0)[0]
freq[20:30]

,Number,Vietnamese,in English
20,21,nhưng,but
21,22,những gì,what
22,23,một số,some
23,24,là,is
24,25,nó,it
25,26,anh,you
26,27,hoặc,or
27,28,có,had
28,29,các,the
29,30,của,of


In [58]:
# TODO: unfinished attempt to split the words
import numpy as np
freq1 = freq.copy()
temp = freq1.Vietnamese.str.split(' ')
freq1 = freq1.reindex(freq1.index.repeat(temp.apply(len)))
freq1.Vietnamese = np.hstack(temp)
freq1.columns = ['number', 'vietnamese', 'english']
# freq1.Vietnamese = freq1.Vietnamese.str.split(" ")
freq1[20:30]

# data = data.reindex(data.index.repeat(temp.apply(len)))
# data['new_Seatblocks'] = np.hstack(temp)

,number,vietnamese,english
20,21,nhưng,but
21,22,những,what
21,22,gì,what
22,23,một,some
22,23,số,some
23,24,là,is
24,25,nó,it
25,26,anh,you
26,27,hoặc,or
27,28,có,had


In [59]:
freq_dict = dict(zip(freq1.vietnamese, freq1.number))

In [60]:
common_vn = []
common_en = []
for i, sentence in enumerate(vn):
  simple = 1
  for word in sentence.split():
    if word in list(freq1.vietnamese):
      pass
    else:
      simple = 0
  if simple:
    common_vn.append(sentence)
    common_en.append(en[i])

print(len(common_vn), len(common_en))

973 973


In [61]:
sentence_values = []
for sentence in common_vn:
  val = 0
  for word in sentence.split():
    word = word.lower()
    val += freq_dict[word]
  avg_val = val / len(sentence)
  sentence_values.append(avg_val)

In [62]:
len(sentence_values)

973

In [63]:
simple_df = pd.DataFrame({'sentence_values': sentence_values, 'common_vn':common_vn, 'common_en':common_en})
simple_df.head(20)

,sentence_values,common_vn,common_en
0,148.080000,sự phát triển của con bạn,your baby s development
1,128.416667,một số trẻ biết nói sớm,some babies talk early
2,151.581395,yếu tố nào xác định được độ chắc của xương,what factors determine bone strength
3,116.461538,giới tính nữ,female gender
4,109.400000,mất kinh tắt kinh,loss of the menstrual period amenorrhea
5,156.310345,cảnh giác với bài tập thể dục,a word of caution about exercise
6,168.857143,cuộc phục kích,ambush
7,158.500000,ở nhiều khu vực khác bề mặt đất chỉ toàn là m...,in some areas the ground is nothing but salt
8,166.522727,thung lũng chết là một khu vực rất rộng lớn,death valley is a huge place
9,126.400000,nhiều người đến tham quan chỉ trong một ngày,many people come for just a day


In [64]:
simple_df.sort_values('sentence_values')

,sentence_values,common_vn,common_en
828,0.000000,,
726,16.000000,và,and
705,39.769231,những tên mới,new entries
778,41.000000,họ tên,full name
539,43.842105,học cách nghiên cứu,learn how to research
...,...,...,...
794,307.333333,đầu cơ,speculation
601,337.800000,tự vệ,selfdefense
140,348.000000,da,skin
137,450.000000,cổ,neck


In [ ]:
out[50]

<spair id="51">
<s id="en51">We had a 3/4 week of it</s>
<s id="vn51">Chúng ta đã có 3/4 tuần</s>
<a id="ev51">1-1,2;2-3,4;4-5;5-6;</a>
</spair>

In [ ]:
pd.read_xml("N0001.sgml")

,title,corpus,author,citation,release,spair
0,What is a Fenqing ?,EVBCorpus,"Quoc-Hung Ngo, Werner Winiwarter","Quoc-Hung Ngo, Werner Winiwarter, 2012. ""Build...",22/10/2013,NaN
1,None,None,None,None,None,NaN
